# Pull/Generate Camera Extrinsics & Intrinsics from Different Sources & Evaluate the Quality of the Extrinsics &Intrinsics Pairs using the BEV Image

In [1]:
%env WAYVE_PRODUCTION=true

env: WAYVE_PRODUCTION=true


In [2]:
from wayve.core.interfaces.protobuf.sensor_calibration_pb2 import CalibrationState, SensorStatus
from wayve.interfaces.protobuf.vehicle_calibration_pb2 import VehicleCalibration
from wayve.core.data.taxonomy import RunId
from wayve.core.calibration.load_calibration import load_calibration, load_calibration_textproto
from wayve.core.calibration.backcompat_support import convert_to_v1
from wayve.core.common.logger import get_logger
from wayve.services.data.lakehouse.pipeline_dag.tables import KEY_CALIBRATION_TABLE
from wayve.services.data.lakehouse.common.external_access.wayve_delta_table import WayveDeltaTable
from wayve.services.calibration.tools.verify_calibration.manual_verification import ManualVerificationManager
from wayve.services.calibration.targetless_extrinsics.pairwise_targetless_extrinsics import generate_pairwise_extrinsics
from wayve.services.calibration.targetless_extrinsics.targetless_calibration_cli.pairwise_calib import _load_merged_df_with_cache
from wayve.core.data.reference_frames import ReferenceFrame, FrameType
from wayve.core.data.cameras_pybindings import DistortionTypeCpp
from wayve.services.data.lakehouse.tools.calibration.load import load_vehicle_calibration_prior_for_run_id
from wayve.services.data.pipelines.spark.common.paths import PROD_PATHS
from pathlib import Path
import glob
import os
import inspect
import pandas as pd
from collections import defaultdict
from typing import Any, Optional, Union

INFO     2024-08-08 00:05:48,836 wayve.core.calibration.utils get_calibration_database_url message=using prod database
INFO     2024-08-08 00:05:48,840 wayve.core.calibration.utils get_calibration_database_url message=using prod database
/home/benjin/.cache/bazel/_bazel_benjin/253920fd9b6cbb686fc69e16c3d2789e/execroot/WayveCode/bazel-out/k8-opt/bin/tools/jupyter_calibration.runfiles/pip-core_pydantic/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_trace" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
INFO     2024-08-08 00:05:54,355 azure.identity._credentials.environment No environment configuration found.


In [3]:
# import importlib
# importlib.reload(manual_verification)
# ManualVerificationManager = manual_verification.ManualVerificationManager

In [4]:
_LOGGER = get_logger(__name__)

RUN_ID = "elbe/2024-07-25--10-07-27--gen2-d97313d5-521a-40c4-a3af-cd9ba31363b4"
OUTPUT_DIR = Path(f"/mnt/remote/data/benjin/bens_notebooks/03_cam_extrinsics_intrinsics_eval/artifacts/bev_imgs/{RUN_ID}/")

COLMAP_MACHES = ["ebro", "elbe", "ganges", "jordan", "mekong", "mono", "napo", "orinoco", "thames"]
RECORDS_DIR = Path("/home/benjin/Development/wcs/WayveCode/wayve/core/calibration/records")
FUNCTIONS_TO_RUN = ["tcs_extrinsics_colmap_intrinsics"]

In [8]:
def generate_bev_overlay_for_run_id_and_calibration(run_id: str, output_directory: Path, calibration: Union[CalibrationState, VehicleCalibration]):
    if isinstance(calibration, VehicleCalibration):
        calibration = convert_to_v1(calibration)
    
    assert isinstance(calibration, CalibrationState)

    cvm = ManualVerificationManager(
        calibration=calibration,
        run_id=RunId.from_string(run_id),
        use_geo_tagging=True,
        ckpt_threshold_m=None,
    )

    cvm._cameras_to_check = set((
        ReferenceFrame.CameraFrontForward, ReferenceFrame.CameraLeftForward, ReferenceFrame.CameraRightForward))

    _ = cvm.manual_verify_bev_overlay(png_dir=output_directory)

def get_colmap_extrinsics_intrinsics(run_id: str) -> CalibrationState:
    """
    The latest records files for our mach-es contain colmap extrinsics and intrinsics
    apart from colorado.
    """
    if isinstance(run_id, str):
        run_id = RunId.from_string(run_id)

    vehicle = run_id.run_vehicle_id

    if vehicle not in COLMAP_MACHES:
        raise ValueError(f"Vehicle {vehicle} did not have colmap extrinsics and intrinsics generated")
    
    # Gets the most recent records calibration for the vehicle that is before the run_datetime
    # Need to take care to not chose a run_id from a mach-e that is before the colmap calibration
    cal_state = load_calibration(vehicle, run_id.run_datetime)

    return cal_state

def get_initial_record_path(vehicle_records_dir: Path) -> Path:
    # Get a list of all files in the directory
    files = glob.glob(os.path.join(vehicle_records_dir.as_posix(), '*'))
    
    # Filter out directories, keeping only files
    files = [f for f in files if os.path.isfile(f)]
    
    # Sort files lexicographically
    files.sort()

    return files[0]

def half_the_resolution(calibration: CalibrationState) -> CalibrationState:
    new_calibration = CalibrationState()
    new_calibration.header.CopyFrom(calibration.header)

    for sensor_status in calibration.state:
        if len(sensor_status.sensors) > 1:
            new_calibration.state.append(sensor_status)
            continue

        if sensor_status.sensors[0] not in ["left-backward", "left-forward", "front-forward", "right-forward", "right-backward"]:
            new_calibration.state.append(sensor_status)
            continue

        # Create a new SensorStatus object and copy the original one
        new_sensor_status = SensorStatus()
        new_sensor_status.CopyFrom(sensor_status)
        
        # Modify the camera_matrix field
        new_sensor_status.camera_matrix[0] /= 2
        new_sensor_status.camera_matrix[2] /= 2
        new_sensor_status.camera_matrix[4] /= 2
        new_sensor_status.camera_matrix[5] /= 2
        
        # Add the modified SensorStatus to the new CalibrationState
        new_calibration.state.append(new_sensor_status)

    return new_calibration

def get_CAD_extrinsics_entron_intrinsics(run_id: str, records_dir: Path) -> CalibrationState:
    """
    Assumption is that the initial records files for our mach-es contain CAD extrinsics and entron intrinsics.
    """
    if isinstance(run_id, str):
        run_id = RunId.from_string(run_id)
    
    vehicle = run_id.run_vehicle_id

    initial_record_path = get_initial_record_path(records_dir / vehicle)

    cad_extrinsics_entron_intrinsics = load_calibration_textproto(Path(initial_record_path))

    # The resolution of the cameras have halved since commissioning for all cameras except left-leftward and right-rightward
    return half_the_resolution(cad_extrinsics_entron_intrinsics)

def get_sensor_status_from_calibration_state(calibration: CalibrationState, sensor_name: str) -> SensorStatus:
    for sensor_status in calibration.state:
        if len(sensor_status.sensors) == 1 and sensor_status.sensors[0] == sensor_name:
            return sensor_status

    raise ValueError(f"Sensor {sensor_name} not found in calibration")

def get_CAD_extrinsics_colmap_intrinsics(run_id: str, records_dir: Path) -> CalibrationState:
    """
    Assumption is that the initial records files for our mach-es contain CAD extrinsics and 
    and that the most recent record files contain colmap intrinsics.
    """
    if isinstance(run_id, str):
        run_id = RunId.from_string(run_id)
    
    cad_extrinsics_entron_intrinsics = get_CAD_extrinsics_entron_intrinsics(run_id, records_dir)
    colmap_extrinsics_intrinsics = get_colmap_extrinsics_intrinsics(run_id)

    new_calibration = CalibrationState()
    new_calibration.header.CopyFrom(cad_extrinsics_entron_intrinsics.header)

    for sensor_status in cad_extrinsics_entron_intrinsics.state:
        if len(sensor_status.sensors) > 1:
            new_calibration.state.append(sensor_status)
            continue

        if sensor_status.sensors[0] not in ["left-backward", "left-forward", "front-forward", 
                                            "right-forward", "right-backward", "left-leftward", "right-rightward"]:
            new_calibration.state.append(sensor_status)
            continue

        intrinsics_frame = sensor_status.sensors[0]

        # Create a new SensorStatus object and copy the original one
        new_sensor_status = SensorStatus()
        new_sensor_status.CopyFrom(sensor_status)
        
        # Replace the entron intrinsics with the colmap inrinsics
        colmap_sensor_status = get_sensor_status_from_calibration_state(colmap_extrinsics_intrinsics, intrinsics_frame)

        # Assert that the camera has not changed over time by checking that the serial numbers are still the same
        assert new_sensor_status.metadata.serial_number == colmap_sensor_status.metadata.serial_number, "Serial number mismatch"

        assert new_sensor_status.camera_geometry == colmap_sensor_status.camera_geometry, "Camera geometry mismatch"
        assert len(new_sensor_status.camera_matrix) == len(colmap_sensor_status.camera_matrix), "Camera matrix size mismatch"
        del new_sensor_status.camera_matrix[:]
        new_sensor_status.camera_matrix.extend(colmap_sensor_status.camera_matrix)

        assert len(new_sensor_status.dist_coeffs) == len(colmap_sensor_status.dist_coeffs), "Distortion coefficients size mismatch"
        del new_sensor_status.dist_coeffs[:]
        new_sensor_status.dist_coeffs.extend(colmap_sensor_status.dist_coeffs)

        resolution_factor = 2
        if intrinsics_frame in ["left-leftward", "right-rightward"]:
            resolution_factor = 1

        assert new_sensor_status.sensor_width_px == colmap_sensor_status.sensor_width_px * resolution_factor, "Sensor width mismatch"
        new_sensor_status.sensor_width_px = colmap_sensor_status.sensor_width_px

        assert new_sensor_status.sensor_height_px == colmap_sensor_status.sensor_height_px * resolution_factor, "Sensor height mismatch"
        new_sensor_status.sensor_height_px = colmap_sensor_status.sensor_height_px
        
        # Add the modified SensorStatus to the new CalibrationState
        new_calibration.state.append(new_sensor_status)

    return new_calibration 

def load_colmap_intrinsics_df(run_id: str) -> pd.DataFrame:
    colmap_extrinsics_intrinsics = get_colmap_extrinsics_intrinsics(run_id)

    intrinsics = defaultdict(list)

    for sensor_status in colmap_extrinsics_intrinsics.state:
        if len(sensor_status.sensors) > 1:
            continue
        
        sensor_name = sensor_status.sensors[0]

        if ReferenceFrame.get_frame_type(sensor_name) != FrameType.Camera:
            continue

        intrinsics["run_id"].append(run_id)
        
        intrinsics["position"].append(sensor_name)

        assert sensor_status.camera_geometry == 2, "Only pinhole equi model is supported"
        intrinsics["distortion_type"].append(str(DistortionTypeCpp.PINHOLE_EQUI))

        intrinsics["fx"].append(sensor_status.camera_matrix[0])
        intrinsics["fy"].append(sensor_status.camera_matrix[4])
        intrinsics["cx"].append(sensor_status.camera_matrix[2])
        intrinsics["cy"].append(sensor_status.camera_matrix[5])

        intrinsics["distortion_parameters"].append(sensor_status.dist_coeffs[:4])

        intrinsics["image_width"].append(sensor_status.sensor_width_px)
        intrinsics["image_height"].append(sensor_status.sensor_height_px)

    intrinsics_df = pd.DataFrame(intrinsics)
    return intrinsics_df

def generate_two_cam_extrinsics(run_id: str) -> pd.DataFrame:
    cameras_to_calibrate = ["front-forward", "left-forward", "right-forward"]
    camera_list = [ReferenceFrame(cam) for cam in cameras_to_calibrate]

    run_df = _load_merged_df_with_cache(run_id)

    vehicle_calibration_prior = load_vehicle_calibration_prior_for_run_id(run_id, PROD_PATHS)

    intrinsics_df = load_colmap_intrinsics_df(run_id)

    output_df = generate_pairwise_extrinsics(
        input_dataframe=run_df,
        vehicle_calibration_prior=vehicle_calibration_prior,
        calibration_df=intrinsics_df,
        cameras_to_calibrate=camera_list,
        images_from_video=True,
    )
    print(output_df.head(100))
    return output_df

def build_tcs_extrinsics_colmap_intrinsics(two_cam_extrinsics_df: pd.DataFrame, run_id: str) -> CalibrationState:
    pass

def get_two_cam_solve_calibration(run_id: str, records_dir: Path) -> CalibrationState:
    # Seed the 2 cam solve with the CAD extrinsics and colmap intrinsics to generate 2-cam extrinsics
    tmp_cache_file = Path("/mnt/remote/data/benjin/bens_notebooks/03_cam_extrinsics_intrinsics_eval/tmp_cache_file.parquet")
    if tmp_cache_file.exists():
        two_cam_extrinsics_df = pd.read_parquet(tmp_cache_file)
    else:
        two_cam_extrinsics_df = generate_two_cam_extrinsics(run_id)
        two_cam_extrinsics_df.to_parquet(tmp_cache_file)

    # Merge two cam extrinsics with colmap intrinsics
    breakpoint()
    tcs_extrinsics_colmap_intrinsics = build_tcs_extrinsics_colmap_intrinsics(two_cam_extrinsics_df, run_id)
    
    return tcs_extrinsics_colmap_intrinsics


calibration_generator_functions = {
    "colmap_extrinsics_intrinsics": get_colmap_extrinsics_intrinsics,
    "cad_extrinsics_entron_intrinsics": get_CAD_extrinsics_entron_intrinsics,
    "cad_extrinsics_colmap_intrinsics": get_CAD_extrinsics_colmap_intrinsics,
    "tcs_extrinsics_colmap_intrinsics": get_two_cam_solve_calibration
}

for func_id, func in calibration_generator_functions.items():
    if func_id not in FUNCTIONS_TO_RUN:
        continue

    if "records_dir" in inspect.signature(func).parameters:
        calibration = func(RUN_ID, RECORDS_DIR)
    else:
        calibration = func(RUN_ID)
    
    generate_bev_overlay_for_run_id_and_calibration(
        RUN_ID, 
        OUTPUT_DIR / func_id, 
        calibration
    )


INFO     2024-08-08 00:17:44,428 wayve.services.calibration.targetless_extrinsics.targetless_calibration_cli.pairwise_calib load_dfs message=loaded dfs from cache
INFO     2024-08-08 00:17:46,344 wayve.services.calibration.targetless_extrinsics.pairwise_targetless_extrinsics find_good_timestamps message=not enough stationary images 🥲 finding the next 36 slowest frames
5it [00:03,  1.43it/s]xtract for camera front-forward:   0%|                                                                             | 0/10 [00:00<?, ?it/s]
5it [00:02,  2.00it/s]xtract for camera front-forward:  10%|██████▉                                                              | 1/10 [00:05<00:50,  5.56s/it]
5it [00:02,  1.73it/s]xtract for camera front-forward:  20%|█████████████▊                                                       | 2/10 [00:09<00:36,  4.53s/it]
5it [00:02,  1.73it/s]xtract for camera front-forward:  30%|████████████████████▋                                                | 3/10 [00:13<00:

Empty DataFrame
Columns: [frame_dest, frame_source, transform, run_id, mean_reprojection_error, number_of_features_for_calibration]
Index: []


AssertionError: 